```{contents}
```
## Branch

A **branch** in LangGraph is a **control-flow construct** that allows execution to **diverge into different paths** based on the current state.
It enables **decision-making, conditional execution, adaptive behavior, and dynamic workflows** in LLM systems.

---

### **1. Why Branching Is Necessary**

Real-world workflows are **not linear**. They require:

| Need                | Role of Branch             |
| ------------------- | -------------------------- |
| Conditional logic   | Choose actions dynamically |
| Error handling      | Recover from failures      |
| Task specialization | Route to expert modules    |
| Autonomy            | Make decisions from state  |
| Scalability         | Avoid monolithic pipelines |

Without branches, a graph is merely a **static pipeline**.

---

### **2. Conceptual Model**

```
                ┌──→ Path A (Search)
Input → Decide ─┤
                └──→ Path B (Compute)
                └──→ Path C (Human Review)
```

Only **one path executes** per decision (unless fan-out is used).

---

### **3. How Branches Are Implemented in LangGraph**

LangGraph implements branching using **conditional edges**.

```python
builder.add_conditional_edges(
    "router_node",
    condition_fn,
    {
        "search": "search_node",
        "compute": "compute_node",
        "human": "human_review_node"
    }
)
```

The `condition_fn` inspects the **state** and returns the key of the next path.

---

### **4. State-Driven Decision Logic**

```python
def route(state):
    if state["needs_search"]:
        return "search"
    if state["confidence"] < 0.7:
        return "human"
    return "compute"
```

This makes branching **deterministic, inspectable, and reproducible**.

---

### **5. Minimal Working Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    x: int

def router(state):
    return "even" if state["x"] % 2 == 0 else "odd"

def even_node(state):
    return {"x": state["x"] * 2}

def odd_node(state):
    return {"x": state["x"] + 1}

builder = StateGraph(State)
builder.add_node("router", router)
builder.add_node("even", even_node)
builder.add_node("odd", odd_node)

builder.set_entry_point("router")

builder.add_conditional_edges("router", router, {
    "even": "even",
    "odd": "odd"
})

builder.add_edge("even", END)
builder.add_edge("odd", END)

graph = builder.compile()
print(graph.invoke({"x": 5}))
```

---

### **6. Common Branching Patterns**

| Pattern          | Purpose               |
| ---------------- | --------------------- |
| If–Else          | Binary decision       |
| Multi-way switch | Task routing          |
| Fallback         | Error recovery        |
| Expert routing   | Specialized handling  |
| Human escalation | Safety and compliance |

---

### **7. Branch vs Loop vs Parallel**

| Feature     | Branch     | Loop        | Parallel      |
| ----------- | ---------- | ----------- | ------------- |
| Paths       | One chosen | Repeats     | Multiple      |
| Use case    | Decision   | Iteration   | Concurrency   |
| Termination | Immediate  | Conditional | Join required |

---

### **8. Production Best Practices**

* Log branch decisions
* Enforce default fallback path
* Limit branch explosion
* Combine with retries & timeouts
* Add human review for high-risk branches

---

### **9. Mental Model**

> **Branch = Decision**
> **Edge = Policy**
> **State = Evidence**

LangGraph branching turns LLM pipelines into **adaptive intelligent systems**.



### Demonstration

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# Step 1 — Define state
class State(TypedDict):
    x: int
    route: str

# Step 2 — Define nodes

def router_node(state: State):
    # Only update state — never return raw values
    return {"route": "even" if state["x"] % 2 == 0 else "odd"}

def even_node(state: State):
    print("Even branch selected")
    return {"x": state["x"] * 2}

def odd_node(state: State):
    print("Odd branch selected")
    return {"x": state["x"] + 1}

# Step 3 — Build graph
builder = StateGraph(State)

builder.add_node("router", router_node)
builder.add_node("even", even_node)
builder.add_node("odd", odd_node)

builder.set_entry_point("router")

# Step 4 — Branch using the state field
def route_decision(state: State):
    return state["route"]

builder.add_conditional_edges(
    "router",
    route_decision,
    {
        "even": "even",
        "odd": "odd"
    }
)

builder.add_edge("even", END)
builder.add_edge("odd", END)

# Step 5 — Compile and run
graph = builder.compile()

print("\nRun with x = 5")
print(graph.invoke({"x": 5}))

print("\nRun with x = 8")
print(graph.invoke({"x": 8}))



Run with x = 5
Odd branch selected
{'x': 6, 'route': 'odd'}

Run with x = 8
Even branch selected
{'x': 16, 'route': 'even'}
